# Spark Files Read/Write ClassTester

Test the DataIO controller that serves as wrapper to read and write data from and to a particular file system.

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True


In [3]:
import os
import sys

proj_dir = os.path.abspath(os.pardir)
sys.path.insert(1,proj_dir.split('rezaware/')[0])
from rezaware.modules.etl.loader import sparkFILEwls as spark
from rezaware.modules.etl.loader.sparkFILEwls import credentials as cred

''' restart initiate classes '''
if debug:
    import importlib
    spark= importlib.reload(spark)

__desc__ = "read and write files from and to a particular source"
clsSpark = spark.FileWorkLoads(desc=__desc__)
if clsSpark._session:
    clsSpark._session.stop
print("\n%s class initialization and load complete!" % __desc__)

All functional SPARKFILEWLS-libraries in LOADER-package of ETL-module imported successfully!
sparkFILEwls Class initialization complete

read and write files from and to a particular source class initialization and load complete!


In [9]:
print(os.environ['SPARK_HOME'])

/opt/spark_hadoop_3/


In [10]:
clsSpark.storeMode = "local-fs"
# clsSpark.storeMode = "aws-s3-bucket"
# clsSpark.storeMode = "google-storage"
print("mode =",clsSpark.storeMode)

''' set the driver '''
if clsSpark.storeMode.lower() == "local-fs":
    clsSpark.jarDir = os.path.join(os.environ['SPARK_HOME'],"jars",
                                   "postgresql-42.6.0.jar")
elif clsSpark.storeMode.lower() == "aws-s3-bucket":
    clsSpark.jarDir = os.path.join(os.environ['SPARK_HOME'],"jars",
                                   "aws-java-sdk-s3-1.12.376.jar")
elif clsSpark.storeMode.lower() == "google-storage":
    clsSpark.jarDir = os.path.join(os.environ['SPARK_HOME'],"jars",
                                   "gcs-connector-hadoop3-2.2.10.jar")
else:
    pass
print("jar =",clsSpark.jarDir)

mode = local-fs
jar = /opt/spark_hadoop_3/jars/postgresql-42.6.0.jar


In [11]:
clsSpark.storeRoot = proj_dir.split('rezaware/')[0]
# clsSpark.storeRoot = "rezaware-wrangler-source-code"
# clsSpark.storeRoot = "tip-daily-marketcap"   #"rezaware-wrangler-source-code"
print("data_root =",clsSpark.storeRoot)

data_root = /home/nuwan/workspace/TIP/


## Read from storage and return as dtype

In [12]:
__as_type__ = "pandas"  # specify data type to return from read file
# folder or key path
__local_folder_path__ = "rezaware/data/etl/loader/sampledata/"
# __aws_folder_path__ = "wrangler/data/hospitality/bookings/scraper/"
# __gcs_folder_path__ = "coingeko"
__local_file_name__ = "sparkfilewls.csv"  #file name to read from or write data to
# __file_name__ = "scraper-build-scrape-url-list.csv" #"room_descriptions.csv"
# __gcs_file_name__ = "2022-11-21.json"  #file name to read from or write data to
__file_type__ = "csv"   # specify file type to write data to or read from
''' options key value pairs must match the pyspark standard'''
options = {
    "inferSchema":True,
    "header":True,
    "delimiter":",",
}

_data = clsSpark.read_files_to_dtype(
    as_type=__as_type__,
    folder_path=__local_folder_path__,
    file_name=None,#__local_file_name__,
    file_type=__file_type__,
    **options,
)
if __as_type__ == 'pandas':
    print("got back {0} rows with return dtype {1}"
          .format(_data.shape[0],__as_type__))
    print(_data.head(5))
elif __as_type__ == 'spark':
    print("got back {0} rows with return dtype {1}"
          .format(_data.count(),__as_type__))
    print(_data.show(5,truncate=False))
elif __as_type__ == 'dict':
    print("got back {0} rows with return dtype {1}"
          .format(len(_data),__as_type__))
    print(_data[:3])
else:
    print('Unrecognised __as_type__' % __as_type__)

/home/nuwan/workspace/TIP/rezaware/data/etl/loader/sampledata/


23/06/14 15:51:33 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
23/06/14 15:51:33 INFO InMemoryFileIndex: It took 3 ms to list leaf files for 1 paths.
23/06/14 15:51:33 INFO FileSourceStrategy: Pushed Filters: 
23/06/14 15:51:33 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#77, None)) > 0)
23/06/14 15:51:33 INFO MemoryStore: Block broadcast_12 stored as values in memory (estimated size 200.2 KiB, free 433.5 MiB)
23/06/14 15:51:33 INFO MemoryStore: Block broadcast_12_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.5 MiB)
23/06/14 15:51:33 INFO BlockManagerInfo: Added broadcast_12_piece0 in memory on 192.168.124.14:39997 (size: 34.5 KiB, free: 434.3 MiB)
23/06/14 15:51:33 INFO SparkContext: Created broadcast 12 from load at NativeMethodAccessorImpl.java:0
23/06/14 15:51:33 INFO FileSourceScanExec: Planning scan with bin packing, max size: 5412341 bytes, open cost is considered as scanning 4194304 bytes.
23/06/14 15:51:33 INFO Sp

23/06/14 15:51:34 INFO FileSourceStrategy: Pushed Filters: 
23/06/14 15:51:34 INFO FileSourceStrategy: Post-Scan Filters: 
23/06/14 15:51:34 INFO MemoryStore: Block broadcast_19 stored as values in memory (estimated size 200.1 KiB, free 433.5 MiB)
23/06/14 15:51:34 INFO MemoryStore: Block broadcast_19_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 433.4 MiB)
23/06/14 15:51:34 INFO BlockManagerInfo: Added broadcast_19_piece0 in memory on 192.168.124.14:39997 (size: 34.5 KiB, free: 434.3 MiB)
23/06/14 15:51:34 INFO SparkContext: Created broadcast 19 from count at NativeMethodAccessorImpl.java:0
23/06/14 15:51:34 INFO FileSourceScanExec: Planning scan with bin packing, max size: 5412341 bytes, open cost is considered as scanning 4194304 bytes.
23/06/14 15:51:34 INFO DAGScheduler: Registering RDD 47 (count at NativeMethodAccessorImpl.java:0) as input to shuffle 3
23/06/14 15:51:34 INFO DAGScheduler: Got map stage job 11 (count at NativeMethodAccessorImpl.java:0) with 1 out

23/06/14 15:51:34 INFO Executor: Finished task 0.0 in stage 17.0 (TID 13). 180482 bytes result sent to driver
23/06/14 15:51:34 INFO TaskSetManager: Finished task 0.0 in stage 17.0 (TID 13) in 199 ms on 192.168.124.14 (executor driver) (1/1)
23/06/14 15:51:34 INFO TaskSchedulerImpl: Removed TaskSet 17.0, whose tasks have all completed, from pool 
23/06/14 15:51:34 INFO DAGScheduler: ResultStage 17 (toPandas at /home/nuwan/workspace/TIP/rezaware/modules/etl/loader/sparkFILEwls.py:1108) finished in 0.210 s
23/06/14 15:51:34 INFO DAGScheduler: Job 13 is finished. Cancelling potential speculative or zombie tasks for this job
23/06/14 15:51:34 INFO TaskSchedulerImpl: Killing all running tasks in stage 17: Stage finished
23/06/14 15:51:34 INFO DAGScheduler: Job 13 finished: toPandas at /home/nuwan/workspace/TIP/rezaware/modules/etl/loader/sparkFILEwls.py:1108, took 0.212777 s


got back 6278 rows with return dtype pandas
   mcap_past_pk                      uuid asset_symbol  mcap_date  \
0         17392  64535d5d4a6e0a5e3a6399cd        1inch 2023-03-01   
1         17393  64535d5d4a6e0a5e3a6399d3        1inch 2023-03-02   
2         17394  64535d5d4a6e0a5e3a6399f2        1inch 2023-03-03   
3         17395  64535d5d4a6e0a5e3a639a0c        1inch 2023-03-04   
4         17396  64535d5d4a6e0a5e3a639a28        1inch 2023-03-05   

     mcap_value asset_name currency              created_dt  created_by  \
0  4.969543e+08      1inch      usd 2023-05-05 07:25:50.866  farmraider   
1  4.915683e+08      1inch      usd 2023-05-05 07:25:50.866  farmraider   
2  4.708599e+08      1inch      usd 2023-05-05 07:25:50.866  farmraider   
3  4.432800e+08      1inch      usd 2023-05-05 07:25:50.866  farmraider   
4  4.400357e+08      1inch      usd 2023-05-05 07:25:50.866  farmraider   

                                        created_proc  
0  wrangler_assets_etl_CryptoMarket

## Filter and save data in storage location

In [13]:
# import pandas as pd

_filter_data = _data.loc[_data['asset_symbol'].isin(['1inch','zks'])]
__file_name__="savetodummy.csv"
print("writing %d filtered rows to file %s in %s" 
      % (_filter_data.shape[0],__file_name__,__local_folder_path__))
write_data=clsSpark.write_data(
    file_name=__file_name__,
    folder_path=__local_folder_path__,
    data=_filter_data
)

print("saved to %s " % write_data)

writing 60 filtered rows to file savetodummy.csv in rezaware/data/etl/loader/sampledata/
saved to <class 'str'> 


In [135]:
from google.cloud import storage


def write_read(bucket_name, blob_name):
    """Write and read a blob from GCS using file-like IO"""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The ID of your new GCS object
    # blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Mode can be specified as wb/rb for bytes mode.
    # See: https://docs.python.org/3/library/io.html
#     with blob.open("w") as f:
#         f.write("Hello world")

    with blob.open("r") as f:
        print(f.read())
